<b> <h5> International Education <h5><b>
In this SQL code, we are an international student interested in taking a master's program either in Business Analytics or Data Science. Using this dataset, we want to figure out which program has the lowest tuition fee and of those programs' location, which would be most affordable to live in.

In [4]:
import pandas as pd
import sqlite3
df_edu = pd.read_csv('/Users/jensa/Downloads/kaggle/International_Education_Costs.csv')

In [5]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://',echo=False)
#Export dataframe to table in sqlite
df_edu.to_sql("edu",con=engine)

907

In [7]:
sql = '''
Select * from edu limit 5

'''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,index,Country,City,University,Program,Level,Duration_Years,Tuition_USD,Living_Cost_Index,Rent_USD,Visa_Fee_USD,Insurance_USD,Exchange_Rate
0,0,USA,Cambridge,Harvard University,Computer Science,Master,2.0,55400,83.5,2200,160,1500,1.00
1,1,UK,London,Imperial College London,Data Science,Master,1.0,41200,75.8,1800,485,800,0.79
2,2,Canada,Toronto,University of Toronto,Business Analytics,Master,2.0,38500,72.5,1600,235,900,1.35
3,3,Australia,Melbourne,University of Melbourne,Engineering,Master,2.0,42000,71.2,1400,450,650,1.52
4,4,Germany,Munich,Technical University of Munich,Mechanical Engineering,Master,2.0,500,70.5,1100,75,550,0.92


In [72]:
#Find out the levels in this data
sql = '''
Select Distinct Level from edu

'''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,Level
0,Master
1,Bachelor
2,PhD


In [152]:
#Our focus are fees incurred. Let's check if the target columns have null values
sql = '''
Select Tuition_USD,Rent_USD,Visa_Fee_USD,Insurance_USD from edu where (Tuition_USD is null or Rent_USD is null or Visa_Fee_USD is null or Insurance_USD is null)

'''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,Tuition_USD,Rent_USD,Visa_Fee_USD,Insurance_USD


In [141]:
#There are no null values for the target columns. Let's compare BA vs DS programs in terms of tuition fee and living cost index.
sql = '''
Select Program,University,Country,City,Tuition_USD,Living_Cost_Index from edu where Level = 'Master' and Program = 'Business Analytics' or Program = 'Data Science' order by Tuition_USD,Living_Cost_Index

'''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head(10)

,Program,University,Country,City,Tuition_USD,Living_Cost_Index
0,Data Science,National University of Cordoba,Argentina,Cordoba,0,39.8
1,Data Science,UFBA,Brazil,Salvador,0,46.8
2,Data Science,UFRGS,Brazil,Porto Alegre,0,49.8
3,Data Science,Athens University of Economics,Greece,Athens,0,65.8
4,Data Science,University of Jyväskylä,Finland,Jyväskylä,0,68.7
5,Data Science,University of Helsinki,Finland,Helsinki,0,79.4
6,Business Analytics,Chamber of Commerce School,Luxembourg,Bertrange,400,75.4
7,Data Science,University of Jena,Germany,Jena,450,63.8
8,Data Science,University of Geneva,Switzerland,Geneva,1380,90.5
9,Data Science,Akdeniz University,Turkey,Antalya,1400,47.8


In [145]:
#Calculate total fees for Data Science program from National University of Cordoba
sql = '''
Select Program,University,Country,City,sum(Tuition_USD+Rent_USD+Visa_Fee_USD+Insurance_USD) as [Total Fees] from edu where Level = 'Master' and Program = 'Data Science' and University = 'National University of Cordoba'

'''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head(10)

,Program,University,Country,City,Total Fees
0,Data Science,National University of Cordoba,Argentina,Cordoba,650


Based on this search, the program above has the lowest tuition fee and lowest living cost index. The total cost to be incurred to enroll in this program is 650 USD.

<b> <h5> In this second part, we will see trends of International Education.

In [92]:
#Average tuition fee for each program level
sql = '''
Select Level,avg(Tuition_USD) from edu group by Level order by avg(Tuition_USD)

'''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head(10)

,Level,avg(Tuition_USD)
0,Master,14521.995565
1,PhD,17129.559748
2,Bachelor,19792.693603


In [125]:
#Top 10 most popular master's programs
sql = '''
Select Program, count(Program) as [Number of Programs] from edu where Level = 'Master' group by Program order by count(Program) desc

'''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head(10)

,Program,Number of Programs
0,Computer Science,114
1,Data Science,57
2,Computer Engineering,37
3,Data Analytics,34
4,Software Engineering,31
5,Artificial Intelligence,30
6,Information Systems,19
7,Data Engineering,15
8,Information Technology,12
9,Software Development,10


In [132]:
#Top 10 highest visa Fees
sql = '''
Select Country,Visa_Fee_USD from edu group by Country order by Visa_Fee_USD desc

'''
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head(10)

,Country,Visa_Fee_USD
0,UK,485
1,Australia,450
2,UAE,270
3,Saudi Arabia,250
4,New Zealand,245
5,Canada,235
6,Japan,220
7,India,200
8,Nigeria,180
9,Netherlands,180
